In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import *
from pyspark.sql.types import *

spark=SparkSession.builder.appName('consultas').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/07 17:37:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/07 17:37:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [24]:
from IPython.core.display import HTML
HTML(r"""
<style>
    .output-plaintext, .output-stream, .output {
        white-space: pre !important;
    }
    pre { white-space: pre !important; }
</style>
""")

In [25]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
spark.conf.set('spark.sql.repl.eagerEval.truncate', 30)
spark.conf.set('spark.sql.repl.eagerEval.maxNumRows', 20)

In [26]:
dataframes_dict = {}
dataframes_dict['metadata_de_facturas']=spark.read.parquet("/Users/tamara/Desktop/Principal/KRANIO/DESAFIO-PYTHON/challenge1/dataset/bkpf_vbrk.parquet")
dataframes_dict['facturas_por_cliente']=spark.read.parquet("/Users/tamara/Desktop/Principal/KRANIO/DESAFIO-PYTHON/challenge1/dataset/vbpa_kna1.parquet")
dataframes_dict['materiales']=spark.read.parquet("/Users/tamara/Desktop/Principal/KRANIO/DESAFIO-PYTHON/challenge1/dataset/vbrp_mara_marm.parquet")
dataframes_dict['paises']=spark.read.parquet("/Users/tamara/Desktop/Principal/KRANIO/DESAFIO-PYTHON/challenge1/dataset/T005T.parquet")

#### Filter Mandante - Idioma Pais

In [27]:
def set_filtros_seleccion_columnas(dataframes_dict):

    df_metadata_facturas = dataframes_dict['metadata_de_facturas']\
                                    .select('Factura', 'Clase_factura', 'Ejercicio', 'Per_contable')\
                                    .where(col('Factura').isNotNull())\
                                    .where(col('Mandante') == '400')

    df_facturas_por_cliente = dataframes_dict['facturas_por_cliente'].select('Factura','Posicion_de_Factura','Pais','Cliente_Destinatario_de_Mercancia','Cliente_solicitante', 'Tipo_de_Cliente')\
                                                .where(col('Cliente_Destinatario_de_Mercancia').isNotNull())\
                                                .withColumnRenamed('Pais', 'Pais_cliente').where(col('Mandante') == '400')

    df_materiales = dataframes_dict['materiales'].select('Factura', 'Posicion', 'Material', 'Unidad_de_medida_Volumen_Destino', 'Volumen').where(col('Mandante') == '400')

    df_paises = dataframes_dict['paises'].select('Idioma', 'Pais' , 'Pais_descr').where(col('Mandante') == '400').where(col('Idioma') == 'S')

    df_resultado = df_materiales.join(df_facturas_por_cliente, 'Factura', 'left')
    df_resultado = df_resultado.join(df_metadata_facturas, 'Factura', 'left')
    df_result = df_result.select('Factura', 'Posicion', 'Clase_factura', 'Tipo_de_Cliente', 'Cliente_solicitante', 'Cliente_Destinatario_de_Mercancia', 'Volumen' )

    return{'tabla_final' : df_resultado}


### Consultas

#### Cantidad de facturas por tipo de cliente

In [28]:
def set_facturas_por_cliente(dataframes_dict):

    df = dataframes_dict['tabla_final']
    df = dataframes_dict['facturas_por_cliente'].groupBy('Tipo_de_Cliente').agg(count('Factura').alias('Cantidad_Facturas')).orderBy(col('Cantidad_Facturas'))

    return{'add_facturas_por_cliente': df}

In [29]:
def main(dataframes_dict):

    functions_list = [
        set_filtros_seleccion_columnas,
        # Otras transformaciones
    ]

    for function in functions_list:
        print(function)
        result_element = function(dataframes_dict)
        dataframes_dict.update(result_element)

    return dataframes_dict['tabla_final'] # Modificar al último dataframe retornado

In [30]:
test = main(dataframes_dict)

<function set_filtros_seleccion_columnas at 0x1223efd00>


In [31]:
test.show(1, False)

+----------+--------+------------------+--------------------------------+-------+-------------------+------------+---------------------------------+-------------------+---------------+-------------+---------+------------+
|Factura   |Posicion|Material          |Unidad_de_medida_Volumen_Destino|Volumen|Posicion_de_Factura|Pais_cliente|Cliente_Destinatario_de_Mercancia|Cliente_solicitante|Tipo_de_Cliente|Clase_factura|Ejercicio|Per_contable|
+----------+--------+------------------+--------------------------------+-------+-------------------+------------+---------------------------------+-------------------+---------------+-------------+---------+------------+
|0150190076|10      |000000000010010108|M3                              |5600.0 |10                 |CL          |164952                           |null               |WE             |YN01         |2020     |12          |
+----------+--------+------------------+--------------------------------+-------+-------------------+-----------

# Consultas

## Consulta 1: Cantidad de facturas por tipo de cliente

In [36]:
dataframes_dict['facturas_por_cliente'].groupBy('Tipo_de_Cliente').agg(count('Factura').alias('Cantidad_Facturas')).orderBy(col('Cantidad_Facturas'))

Tipo_de_Cliente,Cantidad_Facturas
NF,17
SP,196
AV,530
EN,671
SE,1306
VD,2423
RG,2798
VE,2798
RE,2798
AG,2798


## Consulta 2: Top 10 Volumen total por país de cliente destinatario mercancía

In [37]:
dataframes_dict['tabla_final'].groupBy('Pais_cliente', 'Cliente_Destinatario_de_Mercancia').agg(sum('Volumen').alias('Volumen_total')).orderBy(col('Volumen_total').desc()).show(10, False)

+------------+---------------------------------+------------------+
|Pais_cliente|Cliente_Destinatario_de_Mercancia|Volumen_total     |
+------------+---------------------------------+------------------+
|null        |169539                           |446195.5484275913 |
|CL          |243596                           |122000.0          |
|CL          |164952                           |118530.0          |
|US          |C6102                            |10610.667462500058|
|US          |C6108                            |8836.122548012454 |
|US          |C6121                            |8002.88474708978  |
|AE          |156091                           |5495.424999999866 |
|KR          |149510                           |5054.0100000000875|
|AU          |67622                            |4991.908611760056 |
|US          |C6111                            |4834.137287961686 |
+------------+---------------------------------+------------------+
only showing top 10 rows



## Consulta 3: Top 5 Volumen total por cliente solicitante

In [41]:
dataframes_dict['tabla_final'].groupBy('Cliente_solicitante').agg(sum('Volumen').alias('Volumen_total')).orderBy(col('Volumen_total').desc()).show()

+-------------------+-----------------+
|Cliente_solicitante|    Volumen_total|
+-------------------+-----------------+
|               null|899593.4532944837|
+-------------------+-----------------+



## Consulta 4: Volumen total por clase de factura, de mayor a menor volumen

In [40]:
dataframes_dict['tabla_final'].groupBy('Clase_factura').agg(sum('Volumen').alias('Volumen_total')).orderBy(col('Volumen_total').desc()).show(9999999, False)

+-------------+------------------+
|Clase_factura|Volumen_total     |
+-------------+------------------+
|YNDN         |445194.87803089235|
|YN01         |240556.0          |
|YPVE         |103147.76445597172|
|ZINM         |55830.77846639777 |
|YPVN         |47180.698345900484|
|YSDE         |4537.500745486273 |
|YSCE         |1578.4242525546445|
|YSDN         |654.7385285745437 |
|YSCN         |351.38799145984177|
|YPBN         |295.21406943976496|
|YNDE         |266.0684079870466 |
|ZB03         |0.0               |
|ZB13         |0.0               |
|ZB01         |0.0               |
+-------------+------------------+

